In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject\\research'

In [3]:
#we have to be at main/root directory So :

os.chdir("../")  #as we want to create folder artifacts in the main/root directory 

In [4]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject'

#### creating entity : entity is a return type of the function

creating entity

In [7]:
from dataclasses import dataclass  #creating entity using dataclasses
from pathlib import Path


#### dataclasses will work not only as python but here it will work as classes values

In [8]:
@dataclass(frozen=True)  # Frozen:true mean besides these variables , no other variable will be used
class DataIngestionConfig:   # This is entity and this is return type of class this is custom return type can be created with the help of dataclass. (unlike CustomBox)
    root_dir: Path    #root_dir is variable and Path is datatype
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Create Configuration manager

In [21]:
from mlProject.constants import *  # first calling the path of yaml files which are in the constants folder
from mlProject.utils.common import read_yaml, create_directories  # then call the other functions created for reading the yamlfile (read_yaml), and creating dir etc from utils

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  #we store the imported variable in one variable
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath) #read the yaml imported files
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

# using self as it is a class variable
        
        #create a directory using following function
        create_directories([self.config.artifacts_root])

    # 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion  #DataIngestionConfig is the return type. whatever written in dataingestionconfig function ,it will return .You can call this function or entity       config = self.config.data_ingestion

        create_directories([config.root_dir])  #data_ingestion folder should be created inside artifacts. This is code is for that

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,    #one by one we rae returning all the variables 
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

#### Now we are creating components

In [23]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [24]:
class DataIngestion:  
    def __init__(self, config: DataIngestionConfig):  #this DataIngestionConfig has to be passed here so that this function (DataIngestion) can start
        self.config = config


    # First download function from url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):   #if path is not available it will create the path
            filename, headers = request.urlretrieve(   #with the help of request package it will download the data, this will return as a filename and we are storing in local_data_file
                url = self.config.source_URL,
                filename = self.config.local_data_file  
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    # extract the zip file with following function
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir  #provide path
        os.makedirs(unzip_path, exist_ok=True)  
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### Now next from readme file is cretae Pipeline . So we will create the pipeline as below 

In [25]:
try:
    config = ConfigurationManager()  #initialise configurationmanager and created the object ,because  data_ingestion component needs the configuration manager above as config 
    data_ingestion_config = config.get_data_ingestion_config()  #now call get_data_ingestion_config
    data_ingestion = DataIngestion(config=data_ingestion_config) #calling component . Now object is created.
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-11 14:14:45,898: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-11 14:14:45,914: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-11 14:14:45,931: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-11 14:14:45,946: INFO: common: created directory at: artifacts]
[2024-03-11 14:14:45,961: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-11 14:14:47,155: INFO: 676587157: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 368E:0E68:996660:A091FF:65EF03C3
Accept-Ranges: bytes
Date: Mon, 11 Ma

### After runnig this the artifact folder will be created with zip and unzip files

Now convert this notebook into modular coding. Copy paste the code into specific folders 